## 데이터 전처리

In [11]:
import pandas as pd
import numpy as np

stations = pd.read_csv("dataset/usage_stations.csv")
time = pd.read_csv("dataset/usage_time.csv")
day = pd.read_csv("dataset/usage_day.csv")
month = pd.read_csv("dataset/usage_month.csv")

In [12]:
# 필요없는 열 제거 (통계 분석에 무의미한 열 제거)
stations = stations[['gu', 'latitude','longitude','installed','qr','lcd','bike_type']]
"""
대여소 이름도 문자열 처리해서 아파트 앞, 역 앞 등을 확인해서 분석하고 싶지만, geojson 데이터 등
대여소가 이름대로 정말 특정한 시설과 가까운지 확인할 방법이 현재 존재하지 않아 과감하게 제외했습니다.
"""
time = time[[
    'date', 'hour','station_id','rental_type','gender','age','rentals',
    'calories', 'carbon_reduction', 'distance'
]]
day = day[[
    'date','station_id','rental_type','gender','age','rentals',
    'calories', 'carbon_reduction', 'distance', 'time'
]]
month = month[[
    'date','station_id','rental_type','gender','age','rentals',
    'calories', 'carbon_reduction', 'distance', 'time'
]]


In [13]:
# 결측치, 이상치 처리
stations.fillna(0, inplace=True)

for dataset in [time, day, month]:
    dataset['gender'] = dataset['gender'].fillna('unknown')
    dataset['gender'] = dataset['gender'].replace("m", "M")
    dataset['gender'] = dataset['gender'].replace("f", "F")
    dataset[['calories', 'carbon_reduction']] = dataset[['calories', 'carbon_reduction']].fillna(0)

In [14]:
# 결측치 처리 확인
for dataset in [stations, time, day, month]:
    print(dataset.isna().sum())
    print("="*20)

gu           0
latitude     0
longitude    0
installed    0
qr           0
lcd          0
bike_type    0
dtype: int64
date                0
hour                0
station_id          0
rental_type         0
gender              0
age                 0
rentals             0
calories            0
carbon_reduction    0
distance            0
dtype: int64
date                0
station_id          0
rental_type         0
gender              0
age                 0
rentals             0
calories            0
carbon_reduction    0
distance            0
time                0
dtype: int64
date                0
station_id          0
rental_type         0
gender              0
age                 0
rentals             0
calories            0
carbon_reduction    0
distance            0
time                0
dtype: int64


In [15]:
# 시계열 데이터 처리
def add_date_features(df: pd.DataFrame, date_col: str) -> pd.DataFrame:
    """시계열을 년, 월, 일, 요일, 주말 여부로 변환"""
    dt = pd.to_datetime(df[date_col], errors="coerce")

    date_feat = pd.DataFrame({
        f"{date_col}_year": dt.dt.year.astype("Int16"),
        f"{date_col}_month": dt.dt.month.astype("Int8"),
        f"{date_col}_day": dt.dt.day.astype("Int8"),
        f"{date_col}_dow": dt.dt.day_name().astype("category"),
        f"{date_col}_is_weekend": dt.dt.weekday.ge(5).astype("int8"),
    }, index=df.index)

    return df.join(date_feat)

stations = add_date_features(stations, 'installed')
time = add_date_features(time, 'date')
day = add_date_features(day, 'date')


In [16]:
# 가변수화
cat_cols = {
    'stations': ['gu', 'bike_type'],
    'time': ['rental_type', 'gender', 'age'],
    'day': ['rental_type', 'gender', 'age'],
    'month': ['rental_type', 'gender', 'age']
}

stations = pd.get_dummies(stations, columns=cat_cols['stations'])
time = pd.get_dummies(time, columns=cat_cols['time'])
day = pd.get_dummies(day, columns=cat_cols['day'])
month = pd.get_dummies(month, columns=cat_cols['month'])

In [17]:
# 테이블 최종 확인
print(stations.isna().sum())
print(time.isna().sum())
print(day.isna().sum())
print(month.isna().sum())

latitude                0
longitude               0
installed               0
qr                      0
lcd                     0
installed_year          0
installed_month         0
installed_day           0
installed_dow           0
installed_is_weekend    0
gu_강남구                  0
gu_강동구                  0
gu_강북구                  0
gu_강서구                  0
gu_관악구                  0
gu_광진구                  0
gu_구로구                  0
gu_금천구                  0
gu_노원구                  0
gu_도봉구                  0
gu_동대문구                 0
gu_동작구                  0
gu_마포구                  0
gu_서대문구                 0
gu_서초구                  0
gu_성동구                  0
gu_성북구                  0
gu_송파구                  0
gu_양천구                  0
gu_영등포구                 0
gu_용산구                  0
gu_은평구                  0
gu_종로구                  0
gu_중구                   0
gu_중랑구                  0
bike_type_LCD           0
bike_type_LCD,QR        0
bike_type_QR            0
dtype: int64

## 모델링 시작